In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,Kosovo,0.0
2,2020-01-22,no,Kuwait,0.0
3,2020-01-22,no,Kyrgyzstan,0.0
4,2020-01-22,no,Laos,0.0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Spain'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [4]:
test_data.head()

,date,state,country,confirmed
16902,2020-03-21,no,Spain,25374.0
16904,2020-03-21,no,US,26025.0
17185,2020-03-22,no,Spain,28768.0
17203,2020-03-22,no,US,34944.0
17471,2020-03-23,no,Spain,35136.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Spain,2022-07-28,no,13204863.0
US,2022-07-28,no,91120218.0


In [6]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
test_data.groupby(['state','country']).agg(np.max)

date   confirmed
state country                       
no    Spain   2022-07-28  13204863.0
      US      2022-07-28  91120218.0

In [8]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [9]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Spain    16902             NaN
                17185             NaN
                17471        6.096975
                17755        6.224041
                18042        5.773976
                             ...     
       US       260586    4981.487123
                260864     902.088767
                261156     559.789242
                261434     485.521552
                261731     470.547534
Name: confirmed, Length: 1720, dtype: float64

In [10]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR', 'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1048,NaN
4,Alberta,Canada,1354,NaN


In [12]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,Kosovo,0.0
2,2,2020-01-22,no,Kuwait,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,2020-01-22,no,Laos,0.0


In [13]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,Kosovo,0.0,NaN
2,2,2020-01-22,no,Kuwait,0.0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN
4,4,2020-01-22,no,Laos,0.0,NaN


In [14]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [15]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [16]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0
4,4,2020-01-22,no,Laos,0.0,NaN,0.0


In [17]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
261910,no,Zimbabwe,260581,17082.648889
261911,no,Zimbabwe,260823,15530.682828
261912,no,Zimbabwe,261151,13631.556738
261913,no,Zimbabwe,261393,13177.005998
261914,no,Zimbabwe,261914,12263.765550


In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
261910,261910,2022-07-28,no,Andorra,45508.0,498.754579,45435.2,1.247220e+03
261911,261911,2022-07-28,no,Algeria,267194.0,2903.260870,267189.0,3.001124e+03
261912,261912,2022-07-28,no,Albania,309278.0,247.520289,309166.6,2.631772e+02
261913,261913,2022-07-28,no,Argentina,9507562.0,inf,9507562.0,inf
261914,261914,2022-07-28,no,Zimbabwe,256336.0,9858.141026,256333.6,1.226377e+04


In [19]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [20]:
pd_result_larg[pd_result_larg['country']=='Spain'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
260596,260596,2022-07-24,no,Spain,13204863.0,inf,13204536.0,917.433259
260862,260862,2022-07-25,no,Spain,13204863.0,inf,13204209.0,1901.041142
261166,261166,2022-07-26,no,Spain,13203228.0,-1.615207e+04,13203882.0,-40379.844037
261432,261432,2022-07-27,no,Spain,13203228.0,-1.615140e+04,13203391.5,-32302.941896
261713,261713,2022-07-28,no,Spain,13203228.0,inf,13202901.0,-26918.229358


In [21]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)